In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
from colorama import Fore, Style
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, StackingClassifier, RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score, train_test_split, KFold
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import precision_score, recall_score, f1_score, mean_squared_error,  mean_absolute_error, r2_score
from sklearn.inspection import permutation_importance
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)


## Load Data & Select Valid Countries

In [2]:
# Load the data

data_path = os.path.join('..', 'raw_data', 'hotel_bookings_raw.csv')

df = pd.read_csv(data_path)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119390 entries, 0 to 119389
Data columns (total 43 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   hotel                           119390 non-null  object 
 1   is_canceled                     119390 non-null  int64  
 2   lead_time                       119390 non-null  int64  
 3   arrival_date_year               119390 non-null  int64  
 4   arrival_date_month              119390 non-null  object 
 5   arrival_date_week_number        119390 non-null  int64  
 6   arrival_date_day_of_month       119390 non-null  int64  
 7   stays_in_weekend_nights         119390 non-null  int64  
 8   stays_in_week_nights            119390 non-null  int64  
 9   adults                          119390 non-null  int64  
 10  children                        119386 non-null  float64
 11  babies                          119390 non-null  int64  
 12  meal            

In [3]:
#Drop columns
df = df.drop(columns=['arrival_date_year', 'arrival_date_week_number', 'arrival_date_day_of_month', 'children', 'babies', 'previous_cancellations', 'previous_bookings_not_canceled', 'reserved_room_type', 'assigned_room_type', 'booking_changes', 'deposit_type', 'agent', 'days_in_waiting_list', 'customer_type', 'required_car_parking_spaces', 'total_of_special_requests', 'reservation_status_date', 'MO_YR'])
#Drop duplicates
df.drop_duplicates(inplace=True)
#Drop none
df.dropna(inplace=True)

In [4]:
valid_countries = ['PRT', 'GBR', 'ESP', 'FRA', 'DEU']

# Step 3: Filter the DataFrame to only include these valid countries
filtered_df = df[df['country'].isin(valid_countries)]

# Display the filtered DataFrame
# print(f"After removing countries appearing less than {country_threshold} times there are {len(filtered_df)} samples left")
print(f"There are still {len(filtered_df['country'].unique())} countries")
print(filtered_df['country'].unique())

There are still 5 countries
['PRT' 'GBR' 'ESP' 'FRA' 'DEU']


In [5]:
country_percentages = filtered_df['country'].value_counts(normalize=True) * 100

# Display the percentages
print(country_percentages)

country
PRT    45.420270
GBR    17.832893
FRA    15.115074
ESP    12.459796
DEU     9.171968
Name: proportion, dtype: float64


In [6]:
#Drop undefined

filtered_df = filtered_df[filtered_df["meal"] != "Undefined"]
filtered_df = filtered_df[filtered_df["market_segment"] != "Undefined"]
filtered_df = filtered_df[filtered_df["distribution_channel"] != "Undefined"]

print(f"After removing undefined values there are {len(filtered_df)} samples left")

After removing undefined values there are 55593 samples left


In [7]:
# Create the df that we will encode with to not change original data further
encoded_df = filtered_df.copy()

## Mapping & Encoding

In [8]:
#Change months to number
# Create a mapping of month names to numbers
month_mapping = {
    'January': 1,
    'February': 2,
    'March': 3,
    'April': 4,
    'May': 5,
    'June': 6,
    'July': 7,
    'August': 8,
    'September': 9,
    'October': 10,
    'November': 11,
    'December': 12
}
#Strip spaces
encoded_df['arrival_date_month'] = encoded_df['arrival_date_month'].str.strip()
# Replace month names with numbers
encoded_df['arrival_date_month'] = encoded_df['arrival_date_month'].map(month_mapping)

#Change hotel to binary
hotel_mapping = {
    'City Hotel': 1,
    'Resort Hotel': 0
}
#Strip spaces
encoded_df['hotel'] = encoded_df['hotel'].str.strip()
# Replace month names with numbers
encoded_df['hotel'] = encoded_df['hotel'].map(hotel_mapping)

In [9]:
# Change months to numbers
def encode_time(data, col, max_val):
    data[col +'_sin'] = np.sin(2 * np.pi * data[col]/max_val)
    data[col + '_cos'] = np.cos(2 * np.pi * data[col]/max_val)
    return data

encoded_df = encode_time(encoded_df, 'arrival_date_month', 12)

In [10]:
encoded_df.columns

Index(['hotel', 'is_canceled', 'lead_time', 'arrival_date_month',
       'stays_in_weekend_nights', 'stays_in_week_nights', 'adults', 'meal',
       'country', 'market_segment', 'distribution_channel',
       'is_repeated_guest', 'adr', 'reservation_status', 'CPI_AVG',
       'INFLATION', 'INFLATION_CHG', 'CSMR_SENT', 'UNRATE', 'INTRSRT', 'GDP',
       'FUEL_PRCS', 'CPI_HOTELS', 'US_GINI', 'DIS_INC',
       'arrival_date_month_sin', 'arrival_date_month_cos'],
      dtype='object')

In [11]:
# perform 1 hot encoding of categorical values
# One Hot Encode meal, country, market_segment, distribution_channel, reservation_status -> categorical (3-5 categories, encoden)
from sklearn.preprocessing import OneHotEncoder
import numpy as np

# Columns to one-hot encode
# can add 'country' here to use it as a predictor
columns_to_encode = ['meal', 'market_segment', 'distribution_channel', 'reservation_status', 'country']

# Instantiate the OneHotEncoder
ohe = OneHotEncoder(sparse_output=False)  # drop='first' avoids the dummy variable trap

# Fit and transform the data
one_hot_encoded_data = ohe.fit_transform(encoded_df[columns_to_encode])

# Convert encoded data to DataFrame
one_hot_df = pd.DataFrame(one_hot_encoded_data, columns=ohe.get_feature_names_out(columns_to_encode))

# Concatenate the encoded columns with the original DataFrame
encoded_df = pd.concat([encoded_df.reset_index(drop=True), one_hot_df.reset_index(drop=True)], axis=1)

# Drop the original columns that were encoded
encoded_df.drop(columns=columns_to_encode, inplace=True)

# Display the first few rows of the updated DataFrame
print(one_hot_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55593 entries, 0 to 55592
Data columns (total 23 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   meal_BB                         55593 non-null  float64
 1   meal_FB                         55593 non-null  float64
 2   meal_HB                         55593 non-null  float64
 3   meal_SC                         55593 non-null  float64
 4   market_segment_Aviation         55593 non-null  float64
 5   market_segment_Complementary    55593 non-null  float64
 6   market_segment_Corporate        55593 non-null  float64
 7   market_segment_Direct           55593 non-null  float64
 8   market_segment_Groups           55593 non-null  float64
 9   market_segment_Offline TA/TO    55593 non-null  float64
 10  market_segment_Online TA        55593 non-null  float64
 11  distribution_channel_Corporate  55593 non-null  float64
 12  distribution_channel_Direct     

In [12]:
# Robust Scaler
from sklearn.preprocessing import RobustScaler

features_to_robust = ['lead_time', 'stays_in_weekend_nights', 'stays_in_week_nights', 'adults', 'FUEL_PRCS'] #adr out because we want the real numbers
robust_scaler = RobustScaler()
robust_scaler.fit(encoded_df[features_to_robust])
encoded_df[features_to_robust] = robust_scaler.transform(encoded_df[features_to_robust])
encoded_df.head()

,hotel,is_canceled,lead_time,arrival_date_month,stays_in_weekend_nights,stays_in_week_nights,adults,is_repeated_guest,adr,CPI_AVG,...,distribution_channel_GDS,distribution_channel_TA/TO,reservation_status_Canceled,reservation_status_Check-Out,reservation_status_No-Show,country_DEU,country_ESP,country_FRA,country_GBR,country_PRT
0,0,0,2.684685,7,-0.5,-0.666667,0.0,0,0.0,238.034,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
1,0,0,6.243243,7,-0.5,-0.666667,0.0,0,0.0,238.034,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0,0,-0.333333,7,-0.5,-0.333333,-1.0,0,75.0,238.034,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0,0,-0.279279,7,-0.5,-0.333333,-1.0,0,75.0,238.034,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0,0,-0.270270,7,-0.5,0.000000,0.0,0,98.0,238.034,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


In [13]:
from sklearn.preprocessing import MinMaxScaler

features_to_minmax = ['CPI_AVG', 'INFLATION', 'INFLATION_CHG', 'CSMR_SENT', 'UNRATE', 'INTRSRT', 'GDP', 'DIS_INC', 'CPI_HOTELS']
minmax_scaler = MinMaxScaler()
minmax_scaler.fit(encoded_df[features_to_minmax])
encoded_df[features_to_minmax] = minmax_scaler.transform(encoded_df[features_to_minmax])
encoded_df.head()

,hotel,is_canceled,lead_time,arrival_date_month,stays_in_weekend_nights,stays_in_week_nights,adults,is_repeated_guest,adr,CPI_AVG,...,distribution_channel_GDS,distribution_channel_TA/TO,reservation_status_Canceled,reservation_status_Check-Out,reservation_status_No-Show,country_DEU,country_ESP,country_FRA,country_GBR,country_PRT
0,0,0,2.684685,7,-0.5,-0.666667,0.0,0,0.0,0.281229,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
1,0,0,6.243243,7,-0.5,-0.666667,0.0,0,0.0,0.281229,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0,0,-0.333333,7,-0.5,-0.333333,-1.0,0,75.0,0.281229,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0,0,-0.279279,7,-0.5,-0.333333,-1.0,0,75.0,0.281229,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0,0,-0.270270,7,-0.5,0.000000,0.0,0,98.0,0.281229,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


In [14]:
#total stays
encoded_df['total_stays'] = encoded_df['stays_in_weekend_nights'] + encoded_df['stays_in_week_nights']
encoded_df['total_stays']
encoded_df = encoded_df.drop(columns=["stays_in_weekend_nights", "stays_in_week_nights"])


## Model Selection & Training

In [15]:
country_variables = []
for c in encoded_df.columns:
    if "country" in c:
        country_variables.append(c)

In [16]:
# Assuming df_encoded is your DataFrame and adr is the target variable
target_variable = 'adr'
predictors = encoded_df.columns.drop(target_variable)


#if this doesnt work, label encode country first
X = encoded_df[predictors]
y =encoded_df[target_variable]


In [17]:
# train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)


In [18]:
# Train a RandomForestRegressor
#model = LinearRegression()
model=RandomForestRegressor()
model.fit(X_train, y_train)
kf = KFold(n_splits=5, shuffle=True, random_state=42)


In [19]:
#cross val
#cv_scores = cross_val_score(model, X_train, y_train, cv=5, scoring='')
cv_scores = cross_val_score(model, X_train, y_train, cv=kf, scoring='neg_mean_squared_error')

#y_pred = model.predict(X_val)

cv_rmse_scores = np.sqrt(np.abs(cv_scores))

# Print the cross-validation scores
print(f'Cross-Validation RMSE Scores: {cv_rmse_scores}')
print(f'Mean Cross-Validation RMSE: {cv_rmse_scores.mean()}')

# Make predictions on the test set
y_pred = model.predict(X_val)

# Calculate evaluation metrics on the test set
mae = mean_absolute_error(y_val, y_pred)
mse = mean_squared_error(y_val, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_val, y_pred)

print(f'Mean Absolute Error (MAE) on val set: {mae}')
print(f'Mean Squared Error (MSE) on val set: {mse}')
print(f'Root Mean Squared Error (RMSE) on val set: {rmse}')
print(f'R-squared (R2) on val set: {r2}')


#with LinReg:
# Cross-Validation RMSE Scores: [0.59352905 0.57741562 0.58247128 0.57485109 1.06253937]
# Mean Cross-Validation RMSE: 0.678161280251626
# Mean Absolute Error (MAE) on test set: 0.4204011169742372
# Mean Squared Error (MSE) on test set: 0.3488364985245762
# Root Mean Squared Error (RMSE) on test set: 0.590623821501111
# R-squared (R2) on test set: 0.5099760835506549

Cross-Validation RMSE Scores: [29.40719676 28.42955256 28.2285337  28.11210881 68.61127438]
Mean Cross-Validation RMSE: 36.55773324175648
Mean Absolute Error (MAE) on val set: 18.192841752478238
Mean Squared Error (MSE) on val set: 796.8995398321346
Root Mean Squared Error (RMSE) on val set: 28.229409130056805
R-squared (R2) on val set: 0.7267004059997255


In [20]:
from demand_predictor.ml_logic.registry import save_model
save_model(model)

✅ Model saved locally at /Users/mila/code/sarahfink123/demand_prediction/demand_predictor/../training_outputs/models/20240603-152853.pkl


In [21]:

# # Make predictions on the test set
# y_pred = model.predict(X_test)
# mae = mean_absolute_error(y_val, y_pred)
# mse = mean_squared_error(y_val, y_pred)
# rmse = np.sqrt(mse)
# r2 = r2_score(y_val, y_pred)

## Choosing mean/median/mode

In [22]:
# adr features = ['arrival_date_month', 'lead_time', 'stays_in_week_nights', 'adults', 'hotel', 'INFLATION', arrival_date_month]

In [23]:
# means
column_means = encoded_df.drop(columns=["adr"]).mean()
column_means

#pick the variables that we dont in our input and fill the variables of interest with the input of the user

hotel                              0.568039
is_canceled                        0.286691
lead_time                          0.297724
arrival_date_month                 6.512007
adults                            -0.129819
is_repeated_guest                  0.049970
CPI_AVG                            0.530872
INFLATION                          0.638374
INFLATION_CHG                      0.625726
CSMR_SENT                          0.522373
UNRATE                             0.362113
INTRSRT                            0.390796
GDP                                0.530121
FUEL_PRCS                         -0.176606
CPI_HOTELS                         0.534486
US_GINI                           41.150366
DIS_INC                            0.494910
arrival_date_month_sin            -0.049119
arrival_date_month_cos            -0.144101
meal_BB                            0.780692
meal_FB                            0.005576
meal_HB                            0.121886
meal_SC                         

In [24]:
# medians
column_medians = encoded_df.drop(columns=["adr"]).median()
column_medians

#pick the variables that we dont in our input and fill the variables of interest with the input of the user

hotel                             1.000000e+00
is_canceled                       0.000000e+00
lead_time                         0.000000e+00
arrival_date_month                7.000000e+00
adults                            0.000000e+00
is_repeated_guest                 0.000000e+00
CPI_AVG                           4.960643e-01
INFLATION                         7.142857e-01
INFLATION_CHG                     6.666667e-01
CSMR_SENT                         5.221239e-01
UNRATE                            4.285714e-01
INTRSRT                           2.500000e-01
GDP                               4.992595e-01
FUEL_PRCS                         0.000000e+00
CPI_HOTELS                        5.830693e-01
US_GINI                           4.120000e+01
DIS_INC                           4.043584e-01
arrival_date_month_sin           -2.449294e-16
arrival_date_month_cos           -5.000000e-01
meal_BB                           1.000000e+00
meal_FB                           0.000000e+00
meal_HB      

In [25]:
# modes
column_modes = encoded_df.drop(columns=["adr"]).mode()
column_modes

#pick the variables that we dont in our input and fill the variables of interest with the input of the user

,hotel,is_canceled,lead_time,arrival_date_month,adults,is_repeated_guest,CPI_AVG,INFLATION,INFLATION_CHG,CSMR_SENT,...,distribution_channel_TA/TO,reservation_status_Canceled,reservation_status_Check-Out,reservation_status_No-Show,country_DEU,country_ESP,country_FRA,country_GBR,country_PRT,total_stays
0,1,0,-0.396396,8,0.0,0,0.285164,0.857143,0.666667,0.247788,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.833333
